# 전처리 Part

## Import Libraries

In [ ]:
# !pip install pyarrow

In [1]:
import pandas as pd 
import numpy as np
import pyarrow.parquet as pq
from datetime import datetime, timedelta
import datetime as dt

## Import CSV
- 데이터셋 불러오기
  1. 2019년 10월
  2. 2019년 11월

In [2]:
%%time
df_oct = pd.read_csv('../e-commerce/input/2019-Oct.csv')

Wall time: 5min 18s


In [ ]:
%%time
df_nov = pd.read_csv('../e-commerce/input/2019-Nov.csv')

In [3]:
%%time 
#2019년 12월 코호트 분석용 4개 컬럼만 불러옴
df_dec = pd.read_csv('../e-commerce/input/2019-Dec.csv', usecols=['event_time','event_type','user_id','category_code','price'])

Wall time: 1min 37s


In [4]:
%%time
df_jan = pd.read_csv('../e-commerce/input/2020-Jan.csv', usecols=['event_time','event_type','user_id','category_code','price'])

Wall time: 1min 19s


In [5]:
%%time
df_feb = pd.read_csv('../e-commerce/input/2020-Feb.csv', usecols=['event_time','event_type','user_id','category_code','price'])

Wall time: 1min 17s


In [6]:
%%time
df_mar = pd.read_csv('../e-commerce/input/2020-Mar.csv', usecols=['event_time','event_type','user_id','category_code','price'])

Wall time: 1min 12s


In [7]:
%%time
df_apr = pd.read_csv('../e-commerce/input/2020-Apr.csv', usecols=['event_time','event_type','user_id','category_code','price'])

Wall time: 1min 26s


## Export Parquet
- 데이터 압축을 위해 parqeut 자료형으로 내보내기

In [6]:
%%time
df_oct.to_parquet('../e-commerce/input/2019-Oct.parquet', engine='pyarrow', index=False)
df_nov.to_parquet('../e-commerce/input/2019-Nov.parquet', engine='pyarrow', index=False)

Wall time: 4min 49s


In [8]:
%%time
df_dec.to_parquet('../e-commerce/input/2019-Dec.parquet', engine='pyarrow', index=False)

Wall time: 47.5 s


In [9]:
%%time
df_jan.to_parquet('../e-commerce/input/2020-Jan.parquet', engine='pyarrow', index=False)

Wall time: 32.2 s


In [10]:
%%time
df_feb.to_parquet('../e-commerce/input/2020-Feb.parquet', engine='pyarrow', index=False)

Wall time: 31.7 s


In [11]:
%%time
df_mar.to_parquet('../e-commerce/input/2020-Mar.parquet', engine='pyarrow', index=False)

Wall time: 36.4 s


In [12]:
%%time
df_apr.to_parquet('../e-commerce/input/2020-Apr.parquet', engine='pyarrow', index=False)

Wall time: 48.3 s


## Import Parquet
- parqet 불러오기

In [3]:
%%time
df_oct = pd.read_parquet('../e-commerce/input/2019-Oct.parquet')
df_nov = pd.read_parquet('../e-commerce/input/2019-Nov.parquet')

Wall time: 2min 52s


In [13]:
df_dec = pd.read_parquet('../e-commerce/input/2019-Dec.parquet')

In [14]:
df_jan = pd.read_parquet('../e-commerce/input/2020-Jan.parquet')

In [15]:
df_feb = pd.read_parquet('../e-commerce/input/2020-Feb.parquet')

In [16]:
df_mar = pd.read_parquet('../e-commerce/input/2020-Mar.parquet')

In [17]:
df_apr = pd.read_parquet('../e-commerce/input/2020-Apr.parquet')

## Drop category_id
- 불필요한 컬럼 제거 

In [4]:
%%time
df_oct = df_oct.drop('category_id', axis=1)
df_nov = df_nov.drop('category_id', axis=1)

Wall time: 1min 5s


## Missing values
- `user_session` 
  - 10월 : 2개 특성 확인 후 수작업으로 해결
  - 11월 : 10개 특성 확인 후 수작업으로 해결

  - ※ 나중에는 이정도면 그냥 drop 하자
- `category_id`, `category_code`  
  - 'unknown'으로 대체

### Oct.

In [5]:
# 결측치 수 확인
df_oct.isnull().sum()

event_time              0
event_type              0
product_id              0
category_code    13515609
brand             6117080
price                   0
user_id                 0
user_session            2
dtype: int64

In [6]:
# user_session이 결측치인 경우의 user_id 확인 
df_oct[df_oct.user_session.isnull()]

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
7157243,2019-10-06 14:26:10 UTC,cart,1801723,electronics.video.tv,tcl,135.65,557388939,None
33940774,2019-10-25 10:36:14 UTC,cart,1004767,electronics.smartphone,samsung,246.52,549825742,None


### Oct. - Case 1
- 4번->5번
  - view에서 cart로 넘어가면서 session에 오류가 생긴듯 
  - 1번->2번의 경우를 참고하여
  - 5번의 user_session을 4번과 동일하게 설정한다.

In [7]:
df_oct[df_oct.user_id==557388939].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-10-06 14:20:18 UTC,view,50700029,None,yema,48.65,557388939,69e2aec8-4022-45cc-9fb0-48300f6b018e
1,2019-10-06 14:22:05 UTC,view,1801766,electronics.video.tv,artel,150.43,557388939,e24814e3-6587-4f74-8f92-6ac44b9b94ba
2,2019-10-06 14:23:25 UTC,cart,1801766,electronics.video.tv,artel,150.43,557388939,e24814e3-6587-4f74-8f92-6ac44b9b94ba
3,2019-10-06 14:23:50 UTC,view,1801766,electronics.video.tv,artel,150.43,557388939,27eb4e3f-4951-41f5-9ca1-2c8ea537215b
4,2019-10-06 14:25:18 UTC,view,1801723,electronics.video.tv,tcl,135.65,557388939,9c11ac2d-5c56-4994-87e4-ea077ab0ab45
5,2019-10-06 14:26:10 UTC,cart,1801723,electronics.video.tv,tcl,135.65,557388939,None
6,2019-10-07 10:15:39 UTC,view,1801723,electronics.video.tv,tcl,135.65,557388939,083ceaa9-32d0-4e45-a67c-40bfaba0a275


In [8]:
# ndex 값 추출하여 
tmp = df_oct[(df_oct.user_id==557388939) & (df_oct.user_session.isnull())].index[0]

# 4번의 user_session 값으로 대체
df_oct.loc[tmp,'user_session'] = '9c11ac2d-5c56-4994-87e4-ea077ab0ab45'

# 결과 확인
df_oct[df_oct.user_id==557388939].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-10-06 14:20:18 UTC,view,50700029,None,yema,48.65,557388939,69e2aec8-4022-45cc-9fb0-48300f6b018e
1,2019-10-06 14:22:05 UTC,view,1801766,electronics.video.tv,artel,150.43,557388939,e24814e3-6587-4f74-8f92-6ac44b9b94ba
2,2019-10-06 14:23:25 UTC,cart,1801766,electronics.video.tv,artel,150.43,557388939,e24814e3-6587-4f74-8f92-6ac44b9b94ba
3,2019-10-06 14:23:50 UTC,view,1801766,electronics.video.tv,artel,150.43,557388939,27eb4e3f-4951-41f5-9ca1-2c8ea537215b
4,2019-10-06 14:25:18 UTC,view,1801723,electronics.video.tv,tcl,135.65,557388939,9c11ac2d-5c56-4994-87e4-ea077ab0ab45
5,2019-10-06 14:26:10 UTC,cart,1801723,electronics.video.tv,tcl,135.65,557388939,9c11ac2d-5c56-4994-87e4-ea077ab0ab45
6,2019-10-07 10:15:39 UTC,view,1801723,electronics.video.tv,tcl,135.65,557388939,083ceaa9-32d0-4e45-a67c-40bfaba0a275


### Oct. - Case 2
- 크게 2개의 user_seesion이 있다.
  - 같은날의 user_session 값으로 대체 한다. 

In [9]:
df_oct[df_oct.user_id==549825742].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-10-25 09:01:50 UTC,view,1004767,electronics.smartphone,samsung,246.52,549825742,aac8b299-4355-4c25-bbfe-c0369f19a3ce
1,2019-10-25 09:12:12 UTC,cart,1004767,electronics.smartphone,samsung,246.52,549825742,aac8b299-4355-4c25-bbfe-c0369f19a3ce
2,2019-10-25 09:12:42 UTC,cart,1004767,electronics.smartphone,samsung,246.52,549825742,aac8b299-4355-4c25-bbfe-c0369f19a3ce
3,2019-10-25 09:12:52 UTC,cart,1004767,electronics.smartphone,samsung,246.52,549825742,aac8b299-4355-4c25-bbfe-c0369f19a3ce
4,2019-10-25 09:17:05 UTC,cart,1004767,electronics.smartphone,samsung,246.52,549825742,aac8b299-4355-4c25-bbfe-c0369f19a3ce
5,2019-10-25 09:49:33 UTC,view,1004433,electronics.smartphone,samsung,257.15,549825742,56fc550f-2ac9-4baf-a7eb-1d40320560a8
6,2019-10-25 09:50:12 UTC,view,1004434,electronics.smartphone,samsung,257.15,549825742,56fc550f-2ac9-4baf-a7eb-1d40320560a8
7,2019-10-25 09:51:50 UTC,view,1003548,electronics.smartphone,samsung,282.84,549825742,56fc550f-2ac9-4baf-a7eb-1d40320560a8
8,2019-10-25 09:52:44 UTC,view,1000978,electronics.smartphone,samsung,333.49,549825742,56fc550f-2ac9-4baf-a7eb-1d40320560a8
9,2019-10-25 09:53:22 UTC,view,1004990,electronics.smartphone,oppo,257.38,549825742,56fc550f-2ac9-4baf-a7eb-1d40320560a8


In [10]:
# case2의 경우의 index 값 추출하여 
tmp = df_oct[(df_oct.user_id==549825742) & (df_oct.user_session.isnull())].index[0]

# 바로 앞의(9번) user_session 값으로 대체
df_oct.loc[tmp,'user_session'] = '56fc550f-2ac9-4baf-a7eb-1d40320560a8'

# 결과 확인
df_oct[df_oct.user_id==549825742].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-10-25 09:01:50 UTC,view,1004767,electronics.smartphone,samsung,246.52,549825742,aac8b299-4355-4c25-bbfe-c0369f19a3ce
1,2019-10-25 09:12:12 UTC,cart,1004767,electronics.smartphone,samsung,246.52,549825742,aac8b299-4355-4c25-bbfe-c0369f19a3ce
2,2019-10-25 09:12:42 UTC,cart,1004767,electronics.smartphone,samsung,246.52,549825742,aac8b299-4355-4c25-bbfe-c0369f19a3ce
3,2019-10-25 09:12:52 UTC,cart,1004767,electronics.smartphone,samsung,246.52,549825742,aac8b299-4355-4c25-bbfe-c0369f19a3ce
4,2019-10-25 09:17:05 UTC,cart,1004767,electronics.smartphone,samsung,246.52,549825742,aac8b299-4355-4c25-bbfe-c0369f19a3ce
5,2019-10-25 09:49:33 UTC,view,1004433,electronics.smartphone,samsung,257.15,549825742,56fc550f-2ac9-4baf-a7eb-1d40320560a8
6,2019-10-25 09:50:12 UTC,view,1004434,electronics.smartphone,samsung,257.15,549825742,56fc550f-2ac9-4baf-a7eb-1d40320560a8
7,2019-10-25 09:51:50 UTC,view,1003548,electronics.smartphone,samsung,282.84,549825742,56fc550f-2ac9-4baf-a7eb-1d40320560a8
8,2019-10-25 09:52:44 UTC,view,1000978,electronics.smartphone,samsung,333.49,549825742,56fc550f-2ac9-4baf-a7eb-1d40320560a8
9,2019-10-25 09:53:22 UTC,view,1004990,electronics.smartphone,oppo,257.38,549825742,56fc550f-2ac9-4baf-a7eb-1d40320560a8


### Nov.
- 10가지
- 일단 하나하나 채워주긴 했는데 개수가 너무 작아서 전체 데이터에 영향 크게 안줄거 같다.
  - 다음에는 이런 경우 그냥 drop하자

In [11]:
# 결측치 수 확인
df_nov.isnull().sum()

event_time              0
event_type              0
product_id              0
category_code    21898171
brand             9224078
price                   0
user_id                 0
user_session           10
dtype: int64

In [12]:
# user_session이 결측치인 경우의 user_id 확인 
df_nov[df_nov.user_session.isnull()]

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
14771091,2019-11-09 15:32:27 UTC,cart,19700004,None,kabrita,37.77,539704497,None
14991783,2019-11-09 17:15:24 UTC,cart,1005083,electronics.smartphone,honor,566.27,568843390,None
21436266,2019-11-13 04:02:03 UTC,cart,4804008,electronics.audio.headphone,bluedio,97.81,570411102,None
21765703,2019-11-13 07:18:35 UTC,cart,1004767,electronics.smartphone,samsung,243.51,570878749,None
55027396,2019-11-23 12:53:50 UTC,cart,7600528,None,tp-link,16.73,575357602,None
57521839,2019-11-25 05:04:42 UTC,cart,1802104,electronics.video.tv,arg,360.09,573722572,None
57690742,2019-11-25 07:03:38 UTC,cart,21403753,electronics.clocks,None,181.47,576301354,None
59356970,2019-11-26 07:48:12 UTC,cart,4804718,electronics.audio.headphone,apple,334.58,576935861,None
60920894,2019-11-27 07:02:20 UTC,cart,12719553,None,None,55.60,577167045,None
65449633,2019-11-29 17:25:56 UTC,cart,1005116,electronics.smartphone,apple,1001.42,579123407,None


### Nov. - Case 1

In [13]:
df_nov[df_nov.user_id==539704497].reset_index().drop('index', axis=1).head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-09 15:32:06 UTC,view,19700004,None,kabrita,37.77,539704497,e82ecde4-5c22-49e6-84cc-d63def4ff7d3
1,2019-11-09 15:32:27 UTC,cart,19700004,None,kabrita,37.77,539704497,None
2,2019-11-09 15:32:32 UTC,view,19700004,None,kabrita,37.77,539704497,e1e253ec-b564-48a8-8ff5-4e86bacb49f3
3,2019-11-09 15:38:25 UTC,view,19700004,None,kabrita,37.77,539704497,e1e253ec-b564-48a8-8ff5-4e86bacb49f3
4,2019-11-09 15:38:34 UTC,cart,19700004,None,kabrita,37.77,539704497,e1e253ec-b564-48a8-8ff5-4e86bacb49f3


In [14]:
# index 값 추출하여 
tmp = df_nov[(df_nov.user_id==539704497) & (df_nov.user_session.isnull())].index[0]

# 바로 앞의(9번) user_session 값으로 대체
df_nov.loc[tmp,'user_session'] = 'e82ecde4-5c22-49e6-84cc-d63def4ff7d3'

# 결과 확인
df_nov[df_nov.user_id==539704497].reset_index().drop('index', axis=1).head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-09 15:32:06 UTC,view,19700004,None,kabrita,37.77,539704497,e82ecde4-5c22-49e6-84cc-d63def4ff7d3
1,2019-11-09 15:32:27 UTC,cart,19700004,None,kabrita,37.77,539704497,e82ecde4-5c22-49e6-84cc-d63def4ff7d3
2,2019-11-09 15:32:32 UTC,view,19700004,None,kabrita,37.77,539704497,e1e253ec-b564-48a8-8ff5-4e86bacb49f3
3,2019-11-09 15:38:25 UTC,view,19700004,None,kabrita,37.77,539704497,e1e253ec-b564-48a8-8ff5-4e86bacb49f3
4,2019-11-09 15:38:34 UTC,cart,19700004,None,kabrita,37.77,539704497,e1e253ec-b564-48a8-8ff5-4e86bacb49f3


### Nov. - Case 2

In [15]:
df_nov[df_nov.user_id==568843390].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-08 07:14:34 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,ac648108-34a1-418c-b511-a19fc3d29544
1,2019-11-08 09:28:14 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,4b3e9bd9-1325-43e2-9bc5-292be770dc79
2,2019-11-08 12:14:23 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,88c931cc-52c3-450f-a1bb-6d73f54f2bb7
3,2019-11-08 13:35:31 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,7cf58c30-d3be-49b0-bc87-7d25c7b0df8c
4,2019-11-08 14:09:21 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,36949812-f33a-446a-865d-40c83016bac2
5,2019-11-08 15:03:25 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,2199b775-10b8-46c7-b271-3dad0f120156
6,2019-11-09 03:15:37 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,10a156f1-d75b-4e6c-a1f8-d73d97772a0e
7,2019-11-09 03:30:44 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,d4c6c7a4-71d7-4faf-a265-67e7de656bea
8,2019-11-09 07:32:50 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,bbcb9efc-9fa1-427a-9278-2fd6faa1b7a1
9,2019-11-09 09:06:44 UTC,view,1005169,electronics.smartphone,samsung,208.30,568843390,1c395791-cf2e-44ad-b64a-c7e00d05cd0d


In [16]:
# index 값 추출하여 
tmp = df_nov[(df_nov.user_id==568843390) & (df_nov.user_session.isnull())].index[0]

# 바로 앞의(9번) user_session 값으로 대체
df_nov.loc[tmp,'user_session'] = '196aabe5-0f21-4436-b607-b75cf9fceb15'

# 결과 확인
df_nov[df_nov.user_id==568843390].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-08 07:14:34 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,ac648108-34a1-418c-b511-a19fc3d29544
1,2019-11-08 09:28:14 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,4b3e9bd9-1325-43e2-9bc5-292be770dc79
2,2019-11-08 12:14:23 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,88c931cc-52c3-450f-a1bb-6d73f54f2bb7
3,2019-11-08 13:35:31 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,7cf58c30-d3be-49b0-bc87-7d25c7b0df8c
4,2019-11-08 14:09:21 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,36949812-f33a-446a-865d-40c83016bac2
5,2019-11-08 15:03:25 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,2199b775-10b8-46c7-b271-3dad0f120156
6,2019-11-09 03:15:37 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,10a156f1-d75b-4e6c-a1f8-d73d97772a0e
7,2019-11-09 03:30:44 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,d4c6c7a4-71d7-4faf-a265-67e7de656bea
8,2019-11-09 07:32:50 UTC,view,1005169,electronics.smartphone,samsung,208.67,568843390,bbcb9efc-9fa1-427a-9278-2fd6faa1b7a1
9,2019-11-09 09:06:44 UTC,view,1005169,electronics.smartphone,samsung,208.30,568843390,1c395791-cf2e-44ad-b64a-c7e00d05cd0d


### Nov. - Case 3

In [17]:
df_nov[df_nov.user_id==570411102].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-12 05:23:48 UTC,view,4804391,electronics.audio.headphone,sony,200.52,570411102,302ec8bd-146a-4fc1-9cef-558abefd9b88
1,2019-11-12 05:23:53 UTC,view,4804082,electronics.audio.headphone,jbl,120.72,570411102,4b491cfb-4263-400a-b186-56a2f110c887
2,2019-11-12 05:24:43 UTC,view,4803738,electronics.audio.headphone,jbl,84.76,570411102,bf4fe70e-bf35-4d7d-8fba-c32636b2a113
3,2019-11-12 05:26:10 UTC,view,4803580,electronics.audio.headphone,plantronics,141.55,570411102,63d431d4-7f59-45af-afed-c6956ba45da8
4,2019-11-12 05:34:11 UTC,view,4802255,electronics.audio.headphone,marshall,298.59,570411102,a38340d5-d188-49c2-a4ef-b1823a36bd54
5,2019-11-12 05:36:34 UTC,view,4803578,electronics.audio.headphone,plantronics,100.11,570411102,a38340d5-d188-49c2-a4ef-b1823a36bd54
6,2019-11-12 05:36:46 UTC,view,4803580,electronics.audio.headphone,plantronics,141.55,570411102,a38340d5-d188-49c2-a4ef-b1823a36bd54
7,2019-11-12 05:36:55 UTC,view,4803581,electronics.audio.headphone,plantronics,102.71,570411102,a38340d5-d188-49c2-a4ef-b1823a36bd54
8,2019-11-12 05:37:07 UTC,view,4804322,electronics.audio.headphone,plantronics,124.49,570411102,a38340d5-d188-49c2-a4ef-b1823a36bd54
9,2019-11-12 05:38:24 UTC,view,4804447,electronics.audio.headphone,plantronics,100.36,570411102,a38340d5-d188-49c2-a4ef-b1823a36bd54


In [18]:
# index 값 추출하여 
tmp = df_nov[(df_nov.user_id==570411102) & (df_nov.user_session.isnull())].index[0]

# 바로 앞의(9번) user_session 값으로 대체
df_nov.loc[tmp,'user_session'] = '9da8c2c7-5191-4124-9907-83440198093a'

# 결과 확인
df_nov[df_nov.user_id==570411102].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-12 05:23:48 UTC,view,4804391,electronics.audio.headphone,sony,200.52,570411102,302ec8bd-146a-4fc1-9cef-558abefd9b88
1,2019-11-12 05:23:53 UTC,view,4804082,electronics.audio.headphone,jbl,120.72,570411102,4b491cfb-4263-400a-b186-56a2f110c887
2,2019-11-12 05:24:43 UTC,view,4803738,electronics.audio.headphone,jbl,84.76,570411102,bf4fe70e-bf35-4d7d-8fba-c32636b2a113
3,2019-11-12 05:26:10 UTC,view,4803580,electronics.audio.headphone,plantronics,141.55,570411102,63d431d4-7f59-45af-afed-c6956ba45da8
4,2019-11-12 05:34:11 UTC,view,4802255,electronics.audio.headphone,marshall,298.59,570411102,a38340d5-d188-49c2-a4ef-b1823a36bd54
5,2019-11-12 05:36:34 UTC,view,4803578,electronics.audio.headphone,plantronics,100.11,570411102,a38340d5-d188-49c2-a4ef-b1823a36bd54
6,2019-11-12 05:36:46 UTC,view,4803580,electronics.audio.headphone,plantronics,141.55,570411102,a38340d5-d188-49c2-a4ef-b1823a36bd54
7,2019-11-12 05:36:55 UTC,view,4803581,electronics.audio.headphone,plantronics,102.71,570411102,a38340d5-d188-49c2-a4ef-b1823a36bd54
8,2019-11-12 05:37:07 UTC,view,4804322,electronics.audio.headphone,plantronics,124.49,570411102,a38340d5-d188-49c2-a4ef-b1823a36bd54
9,2019-11-12 05:38:24 UTC,view,4804447,electronics.audio.headphone,plantronics,100.36,570411102,a38340d5-d188-49c2-a4ef-b1823a36bd54


### Nov. - Case 4

In [19]:
df_nov[df_nov.user_id==570878749].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-13 07:17:41 UTC,view,1004767,electronics.smartphone,samsung,243.51,570878749,836c4aa8-e371-4033-82ab-b44b4de7ce5b
1,2019-11-13 07:18:35 UTC,cart,1004767,electronics.smartphone,samsung,243.51,570878749,None


In [20]:
# index 값 추출하여 
tmp = df_nov[(df_nov.user_id==570878749) & (df_nov.user_session.isnull())].index[0]

# 바로 앞의(9번) user_session 값으로 대체
df_nov.loc[tmp,'user_session'] = '836c4aa8-e371-4033-82ab-b44b4de7ce5b'

# 결과 확인
df_nov[df_nov.user_id==570878749].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-13 07:17:41 UTC,view,1004767,electronics.smartphone,samsung,243.51,570878749,836c4aa8-e371-4033-82ab-b44b4de7ce5b
1,2019-11-13 07:18:35 UTC,cart,1004767,electronics.smartphone,samsung,243.51,570878749,836c4aa8-e371-4033-82ab-b44b4de7ce5b


### Nov. - Case 5

In [21]:
df_nov[df_nov.user_id==575357602].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-23 09:52:02 UTC,view,7600251,None,d-link,15.93,575357602,cecc5635-dc3f-4496-bc77-9c403022161b
1,2019-11-23 09:53:13 UTC,view,7600252,None,tp-link,23.29,575357602,57fac5f8-cf77-4557-a135-9589e31ffc47
2,2019-11-23 09:54:40 UTC,view,7600528,None,tp-link,16.73,575357602,a052d1fc-5c27-44c7-964a-97808573d88d
3,2019-11-23 09:56:57 UTC,view,7600253,None,d-link,20.81,575357602,0a9691f8-56db-4d5a-9e7e-f37f51767a43
4,2019-11-23 09:58:47 UTC,view,7600323,None,d-link,23.02,575357602,bf6a8130-72f8-42e1-a7f0-9e904139b703
5,2019-11-23 12:53:50 UTC,cart,7600528,None,tp-link,16.73,575357602,None


In [22]:
# index 값 추출하여 
tmp = df_nov[(df_nov.user_id==575357602) & (df_nov.user_session.isnull())].index[0]

# 바로 앞의(9번) user_session 값으로 대체
df_nov.loc[tmp,'user_session'] = 'bf6a8130-72f8-42e1-a7f0-9e904139b703'

# 결과 확인
df_nov[df_nov.user_id==575357602].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-23 09:52:02 UTC,view,7600251,None,d-link,15.93,575357602,cecc5635-dc3f-4496-bc77-9c403022161b
1,2019-11-23 09:53:13 UTC,view,7600252,None,tp-link,23.29,575357602,57fac5f8-cf77-4557-a135-9589e31ffc47
2,2019-11-23 09:54:40 UTC,view,7600528,None,tp-link,16.73,575357602,a052d1fc-5c27-44c7-964a-97808573d88d
3,2019-11-23 09:56:57 UTC,view,7600253,None,d-link,20.81,575357602,0a9691f8-56db-4d5a-9e7e-f37f51767a43
4,2019-11-23 09:58:47 UTC,view,7600323,None,d-link,23.02,575357602,bf6a8130-72f8-42e1-a7f0-9e904139b703
5,2019-11-23 12:53:50 UTC,cart,7600528,None,tp-link,16.73,575357602,bf6a8130-72f8-42e1-a7f0-9e904139b703


### Nov. - Case 6

In [23]:
df_nov[df_nov.user_id==573722572].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-20 03:30:19 UTC,view,1201216,electronics.tablet,lenovo,244.25,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
1,2019-11-20 03:30:57 UTC,view,1201216,electronics.tablet,lenovo,244.25,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
2,2019-11-20 03:31:37 UTC,view,1200965,electronics.tablet,apple,1724.60,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
3,2019-11-20 03:33:57 UTC,view,1200965,electronics.tablet,apple,1724.60,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
4,2019-11-20 03:39:56 UTC,view,1004741,electronics.smartphone,xiaomi,188.42,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
5,2019-11-20 03:40:08 UTC,view,1004741,electronics.smartphone,xiaomi,188.42,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
6,2019-11-20 03:40:58 UTC,view,1004741,electronics.smartphone,xiaomi,188.42,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
7,2019-11-20 03:41:06 UTC,cart,1004741,electronics.smartphone,xiaomi,188.42,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
8,2019-11-20 03:41:29 UTC,view,1004741,electronics.smartphone,xiaomi,188.42,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
9,2019-11-20 03:41:31 UTC,cart,1004741,electronics.smartphone,xiaomi,188.42,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a


In [24]:
# index 값 추출하여 
tmp = df_nov[(df_nov.user_id==573722572) & (df_nov.user_session.isnull())].index[0]

# 바로 앞의(9번) user_session 값으로 대체
df_nov.loc[tmp,'user_session'] = 'd3835e60-b927-4ad8-bf21-fed6a905e84e'

# 결과 확인
df_nov[df_nov.user_id==573722572].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-20 03:30:19 UTC,view,1201216,electronics.tablet,lenovo,244.25,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
1,2019-11-20 03:30:57 UTC,view,1201216,electronics.tablet,lenovo,244.25,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
2,2019-11-20 03:31:37 UTC,view,1200965,electronics.tablet,apple,1724.60,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
3,2019-11-20 03:33:57 UTC,view,1200965,electronics.tablet,apple,1724.60,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
4,2019-11-20 03:39:56 UTC,view,1004741,electronics.smartphone,xiaomi,188.42,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
5,2019-11-20 03:40:08 UTC,view,1004741,electronics.smartphone,xiaomi,188.42,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
6,2019-11-20 03:40:58 UTC,view,1004741,electronics.smartphone,xiaomi,188.42,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
7,2019-11-20 03:41:06 UTC,cart,1004741,electronics.smartphone,xiaomi,188.42,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
8,2019-11-20 03:41:29 UTC,view,1004741,electronics.smartphone,xiaomi,188.42,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a
9,2019-11-20 03:41:31 UTC,cart,1004741,electronics.smartphone,xiaomi,188.42,573722572,681df41a-8b44-4569-b8a8-aeed7a9eae1a


### Nov. - Case 7

In [25]:
df_nov[df_nov.user_id==576301354].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-25 06:09:14 UTC,view,21404608,electronics.clocks,None,262.30,576301354,c38c005b-8cbc-4a21-ab3d-9763db74f932
1,2019-11-25 06:14:54 UTC,view,21403753,electronics.clocks,None,181.47,576301354,c38c005b-8cbc-4a21-ab3d-9763db74f932
2,2019-11-25 06:15:50 UTC,view,21404608,electronics.clocks,None,262.30,576301354,c38c005b-8cbc-4a21-ab3d-9763db74f932
3,2019-11-25 06:16:16 UTC,view,21403753,electronics.clocks,None,181.47,576301354,c38c005b-8cbc-4a21-ab3d-9763db74f932
4,2019-11-25 07:03:38 UTC,cart,21403753,electronics.clocks,None,181.47,576301354,None


In [26]:
# index 값 추출하여 
tmp = df_nov[(df_nov.user_id==576301354) & (df_nov.user_session.isnull())].index[0]

# 바로 앞의(9번) user_session 값으로 대체
df_nov.loc[tmp,'user_session'] = 'c38c005b-8cbc-4a21-ab3d-9763db74f932'

# 결과 확인
df_nov[df_nov.user_id==576301354].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-25 06:09:14 UTC,view,21404608,electronics.clocks,None,262.30,576301354,c38c005b-8cbc-4a21-ab3d-9763db74f932
1,2019-11-25 06:14:54 UTC,view,21403753,electronics.clocks,None,181.47,576301354,c38c005b-8cbc-4a21-ab3d-9763db74f932
2,2019-11-25 06:15:50 UTC,view,21404608,electronics.clocks,None,262.30,576301354,c38c005b-8cbc-4a21-ab3d-9763db74f932
3,2019-11-25 06:16:16 UTC,view,21403753,electronics.clocks,None,181.47,576301354,c38c005b-8cbc-4a21-ab3d-9763db74f932
4,2019-11-25 07:03:38 UTC,cart,21403753,electronics.clocks,None,181.47,576301354,c38c005b-8cbc-4a21-ab3d-9763db74f932


### Nov. - Case 8

In [27]:
df_nov[df_nov.user_id==576935861].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-26 07:47:29 UTC,view,4804718,electronics.audio.headphone,apple,334.58,576935861,5ac42010-7cb2-4151-b4d4-840fbd5c3102
1,2019-11-26 07:48:12 UTC,cart,4804718,electronics.audio.headphone,apple,334.58,576935861,None
2,2019-11-29 13:31:39 UTC,view,1004659,electronics.smartphone,samsung,738.35,576935861,670cd99b-3c5b-42f7-9b97-5f0520dc561a


In [28]:
# index 값 추출하여 
tmp = df_nov[(df_nov.user_id==576935861) & (df_nov.user_session.isnull())].index[0]

# 바로 앞의(9번) user_session 값으로 대체
df_nov.loc[tmp,'user_session'] = '5ac42010-7cb2-4151-b4d4-840fbd5c3102'

# 결과 확인
df_nov[df_nov.user_id==576935861].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-26 07:47:29 UTC,view,4804718,electronics.audio.headphone,apple,334.58,576935861,5ac42010-7cb2-4151-b4d4-840fbd5c3102
1,2019-11-26 07:48:12 UTC,cart,4804718,electronics.audio.headphone,apple,334.58,576935861,5ac42010-7cb2-4151-b4d4-840fbd5c3102
2,2019-11-29 13:31:39 UTC,view,1004659,electronics.smartphone,samsung,738.35,576935861,670cd99b-3c5b-42f7-9b97-5f0520dc561a


### Nov. - Case 9

In [29]:
df_nov[df_nov.user_id==577167045].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-26 15:46:37 UTC,view,12709269,None,triangle,55.34,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
1,2019-11-26 15:46:59 UTC,view,12709269,None,triangle,55.34,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
2,2019-11-26 15:47:26 UTC,view,12719553,None,None,55.60,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
3,2019-11-26 15:48:58 UTC,view,12717696,None,pirelli,72.72,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
4,2019-11-26 15:49:38 UTC,view,100000237,None,roadmarch,56.55,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
5,2019-11-26 15:49:56 UTC,view,12720098,None,roadx,48.91,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
6,2019-11-26 15:50:20 UTC,view,12719553,None,None,55.60,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
7,2019-11-26 15:50:32 UTC,cart,12719553,None,None,55.60,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
8,2019-11-26 15:59:14 UTC,view,12719553,None,None,55.60,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
9,2019-11-26 15:59:24 UTC,view,12719553,None,None,55.60,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3


In [30]:
# index 값 추출하여 
tmp = df_nov[(df_nov.user_id==577167045) & (df_nov.user_session.isnull())].index[0]

# 바로 앞의(9번) user_session 값으로 대체
df_nov.loc[tmp,'user_session'] = '1dcf3c18-66e7-4a23-8e55-1ab390258607'

# 결과 확인
df_nov[df_nov.user_id==577167045].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-26 15:46:37 UTC,view,12709269,None,triangle,55.34,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
1,2019-11-26 15:46:59 UTC,view,12709269,None,triangle,55.34,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
2,2019-11-26 15:47:26 UTC,view,12719553,None,None,55.60,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
3,2019-11-26 15:48:58 UTC,view,12717696,None,pirelli,72.72,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
4,2019-11-26 15:49:38 UTC,view,100000237,None,roadmarch,56.55,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
5,2019-11-26 15:49:56 UTC,view,12720098,None,roadx,48.91,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
6,2019-11-26 15:50:20 UTC,view,12719553,None,None,55.60,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
7,2019-11-26 15:50:32 UTC,cart,12719553,None,None,55.60,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
8,2019-11-26 15:59:14 UTC,view,12719553,None,None,55.60,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3
9,2019-11-26 15:59:24 UTC,view,12719553,None,None,55.60,577167045,71c1ca1d-c087-43bb-a9a5-785fc557bfc3


### Nov. - Case 10

In [31]:
df_nov[df_nov.user_id==579123407].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-29 17:21:54 UTC,view,1005116,electronics.smartphone,apple,1001.42,579123407,a8d85676-92e4-457c-b705-69e907217ca6
1,2019-11-29 17:25:56 UTC,cart,1005116,electronics.smartphone,apple,1001.42,579123407,None


In [32]:
# index 값 추출하여 
tmp = df_nov[(df_nov.user_id==579123407) & (df_nov.user_session.isnull())].index[0]

# 바로 앞의(9번) user_session 값으로 대체
df_nov.loc[tmp,'user_session'] = 'a8d85676-92e4-457c-b705-69e907217ca6'

# 결과 확인
df_nov[df_nov.user_id==579123407].reset_index().drop('index', axis=1)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session
0,2019-11-29 17:21:54 UTC,view,1005116,electronics.smartphone,apple,1001.42,579123407,a8d85676-92e4-457c-b705-69e907217ca6
1,2019-11-29 17:25:56 UTC,cart,1005116,electronics.smartphone,apple,1001.42,579123407,a8d85676-92e4-457c-b705-69e907217ca6


### category_code, brand

- 2019.10~11

In [33]:
df_oct = df_oct.fillna('unknown')
df_nov = df_nov.fillna('unknown')

In [34]:
df_oct.isnull().sum()

event_time       0
event_type       0
product_id       0
category_code    0
brand            0
price            0
user_id          0
user_session     0
dtype: int64

In [35]:
df_nov.isnull().sum()

event_time       0
event_type       0
product_id       0
category_code    0
brand            0
price            0
user_id          0
user_session     0
dtype: int64

- 2019.12~2020.4

In [25]:
df_dec.isnull().sum()

event_time             0
event_type             0
category_code    7088848
user_id                0
dtype: int64

In [18]:
df_dec = df_dec.fillna('unknown')
df_jan = df_jan.fillna('unknown')
df_feb = df_feb.fillna('unknown')
df_mar = df_mar.fillna('unknown')
df_apr = df_apr.fillna('unknown')

## Change dtypes
1. `event_time` : datetime 형태로 변경
2. 불필요한 메모리 축소
  1. `event_type`, `category_code`, `brand`, `user_session` : object -> category
    - (주의) 카테고리 변환이후 카테고리 수정시 에러발생, 
    - object 형태에서 모든 전처리 후 형변환 진행할 것
  2. `product_id`, `user_id` : int64 -> int32
  3. `price` : float64 -> float32

In [36]:
df_oct.dtypes   # 컬럼별 type 확인하기

event_time        object
event_type        object
product_id         int64
category_code     object
brand             object
price            float64
user_id            int64
user_session      object
dtype: object

In [37]:
df_nov.dtypes

event_time        object
event_type        object
product_id         int64
category_code     object
brand             object
price            float64
user_id            int64
user_session      object
dtype: object

In [38]:
%time
# 1. datetime 형변환
df_oct['event_time'] = pd.to_datetime(df_oct['event_time'], format = '%Y-%m-%d %H:%M:%S UTC')
df_nov['event_time'] = pd.to_datetime(df_nov['event_time'], format = '%Y-%m-%d %H:%M:%S UTC')

Wall time: 0 ns


In [39]:
%%time
# 2. 기타 형변환
# category 변환
df_oct[['event_type','category_code','brand','user_session']] = df_oct[['event_type','category_code','brand','user_session']].astype('category')
df_nov[['event_type','category_code','brand','user_session']] = df_nov[['event_type','category_code','brand','user_session']].astype('category')

# int 변환
df_oct[['product_id','user_id']] = df_oct[['product_id','user_id']].astype('int32')
df_nov[['product_id','user_id']] = df_nov[['product_id','user_id']].astype('int32')

# float 변환
df_oct['price'] = df_oct['price'].astype('float32')
df_nov['price'] = df_nov['price'].astype('float32')

Wall time: 3min 19s


In [40]:
df_oct.dtypes 

event_time       datetime64[ns]
event_type             category
product_id                int32
category_code          category
brand                  category
price                   float32
user_id                   int32
user_session           category
dtype: object

In [41]:
df_nov.dtypes

event_time       datetime64[ns]
event_type             category
product_id                int32
category_code          category
brand                  category
price                   float32
user_id                   int32
user_session           category
dtype: object

-2019년 12월~2020년 4월

In [20]:
df_dec.dtypes

event_time        object
event_type        object
category_code     object
price            float64
user_id            int64
dtype: object

In [22]:
def cleaning_dtypes(df):
    # 1. datetime 형변환
    df['event_time'] = pd.to_datetime(df['event_time'], format = '%Y-%m-%d %H:%M:%S UTC')
    
    # 2. 기타 형변환
    # category 변환
    df[['event_type','category_code']] = df[['event_type','category_code']].astype('category')

    # int 변환
    df[['user_id']] = df[['user_id']].astype('int32')
    
    # float 변환
    df['price'] = df['price'].astype('float32')
    
    
    return df

In [23]:
%%time
cleaning_dtypes(df_dec)

Wall time: 31.4 s


,event_time,event_type,category_code,price,user_id
0,2019-12-01 00:00:00,view,construction.tools.light,1302.479980,556695836
1,2019-12-01 00:00:00,view,unknown,102.959999,577702456
2,2019-12-01 00:00:01,view,appliances.personal.massager,313.519989,539453785
3,2019-12-01 00:00:02,purchase,computers.peripherals.printer,132.309998,535135317
4,2019-12-01 00:00:02,view,apparel.trousers,101.680000,517987650
...,...,...,...,...,...
67542873,2019-12-31 23:59:58,view,construction.tools.light,908.799988,515771654
67542874,2019-12-31 23:59:58,view,electronics.audio.headphone,1044.250000,595160289
67542875,2019-12-31 23:59:59,view,kids.dolls,411.829987,526621231
67542876,2019-12-31 23:59:59,view,sport.bicycle,17.990000,574723072


In [24]:
cleaning_dtypes(df_jan)

,event_time,event_type,category_code,price,user_id
0,2020-01-01 00:00:00,view,construction.tools.light,1130.020020,519698804
1,2020-01-01 00:00:01,view,construction.tools.light,205.669998,527767423
2,2020-01-01 00:00:01,view,apparel.shirt,136.429993,519046195
3,2020-01-01 00:00:01,view,apparel.shoes,29.950001,518269232
4,2020-01-01 00:00:02,view,apparel.shoes.step_ins,167.199997,587748686
...,...,...,...,...,...
55967036,2020-01-31 23:59:59,view,apparel.shoes,33.720001,517548219
55967037,2020-01-31 23:59:59,view,construction.tools.light,228.830002,512985520
55967038,2020-01-31 23:59:59,view,apparel.shoes,47.360001,513043451
55967039,2020-01-31 23:59:59,view,furniture.bedroom.blanket,51.299999,581011571


In [25]:
cleaning_dtypes(df_feb)

,event_time,event_type,category_code,price,user_id
0,2020-02-01 00:00:00,view,apparel.shoes,19.309999,581521975
1,2020-02-01 00:00:00,view,sport.bicycle,11.880000,596313635
2,2020-02-01 00:00:00,view,construction.tools.light,838.229980,608821035
3,2020-02-01 00:00:01,view,construction.tools.light,410.420013,530488542
4,2020-02-01 00:00:01,view,appliances.kitchen.refrigerators,332.029999,583795643
...,...,...,...,...,...
55318560,2020-02-29 23:59:57,view,construction.tools.welding,1.930000,519011545
55318561,2020-02-29 23:59:58,view,appliances.kitchen.toster,77.199997,540823940
55318562,2020-02-29 23:59:58,view,electronics.telephone,49.910000,531574188
55318563,2020-02-29 23:59:59,view,electronics.camera.video,10.770000,614934914


In [26]:
cleaning_dtypes(df_mar)

,event_time,event_type,category_code,price,user_id
0,2020-03-01 00:00:00,view,appliances.kitchen.mixer,117.629997,550295099
1,2020-03-01 00:00:01,view,apparel.underwear,264.369995,622070532
2,2020-03-01 00:00:03,view,furniture.bedroom.bed,126.099998,552832344
3,2020-03-01 00:00:03,view,construction.tools.generator,424.720001,613391860
4,2020-03-01 00:00:04,view,construction.tools.light,482.730011,517021211
...,...,...,...,...,...
56341236,2020-03-31 23:59:55,view,construction.tools.light,178.119995,516506125
56341237,2020-03-31 23:59:55,view,appliances.personal.massager,154.419998,572260610
56341238,2020-03-31 23:59:56,view,unknown,167.289993,523607693
56341239,2020-03-31 23:59:56,view,appliances.personal.massager,163.160004,629229484


In [27]:
cleaning_dtypes(df_apr)

,event_time,event_type,category_code,price,user_id
0,2020-04-01 00:00:00,view,apparel.shoes.slipons,230.380005,568984877
1,2020-04-01 00:00:01,view,electronics.audio.headphone,1352.670044,514955500
2,2020-04-01 00:00:01,view,appliances.kitchen.refrigerators,1184.050049,633645770
3,2020-04-01 00:00:02,view,electronics.audio.headphone,1724.339966,564933778
4,2020-04-01 00:00:03,view,apparel.scarf,25.049999,530206135
...,...,...,...,...,...
66589263,2020-04-30 23:59:59,view,construction.tools.light,900.900024,648021639
66589264,2020-04-30 23:59:59,view,construction.tools.light,1599.780029,649775363
66589265,2020-04-30 23:59:59,view,apparel.shirt,12.840000,639250275
66589266,2020-04-30 23:59:59,view,appliances.kitchen.toster,1956.030029,522762529


## event_time 관련 특성공학
- GMT+6(카자흐스탄으로 가정) 
- 년월일, 월, 일, 요일, 시간, 주차 특성 생성

In [42]:
# 10월
df_oct['event_time'].head()

0   2019-10-01 00:00:00
1   2019-10-01 00:00:00
2   2019-10-01 00:00:01
3   2019-10-01 00:00:01
4   2019-10-01 00:00:04
Name: event_time, dtype: datetime64[ns]

In [43]:
# timedelta로 전체 시간 +6
df_oct['event_time'] = pd.DatetimeIndex(df_oct['event_time']) + timedelta(hours=6)
df_oct['event_time']

0          2019-10-01 06:00:00
1          2019-10-01 06:00:00
2          2019-10-01 06:00:01
3          2019-10-01 06:00:01
4          2019-10-01 06:00:04
                   ...        
42448759   2019-11-01 05:59:58
42448760   2019-11-01 05:59:58
42448761   2019-11-01 05:59:58
42448762   2019-11-01 05:59:59
42448763   2019-11-01 05:59:59
Name: event_time, Length: 42448764, dtype: datetime64[ns]

In [44]:
# 11월
df_nov['event_time'].head()

0   2019-11-01 00:00:00
1   2019-11-01 00:00:00
2   2019-11-01 00:00:01
3   2019-11-01 00:00:01
4   2019-11-01 00:00:01
Name: event_time, dtype: datetime64[ns]

In [45]:
# timedelta로 전체 시간 +6 
df_nov['event_time'] = pd.DatetimeIndex(df_nov['event_time']) + timedelta(hours=6)
df_nov['event_time']

0          2019-11-01 06:00:00
1          2019-11-01 06:00:00
2          2019-11-01 06:00:01
3          2019-11-01 06:00:01
4          2019-11-01 06:00:01
                   ...        
67501974   2019-12-01 05:59:58
67501975   2019-12-01 05:59:58
67501976   2019-12-01 05:59:59
67501977   2019-12-01 05:59:59
67501978   2019-12-01 05:59:59
Name: event_time, Length: 67501979, dtype: datetime64[ns]

In [46]:
%%time
# 날짜만 별도로 추출
df_oct['event_date'] = pd.DatetimeIndex(df_oct['event_time']).date
df_nov['event_date'] = pd.DatetimeIndex(df_nov['event_time']).date
df_oct.head(1)

Wall time: 35.6 s


,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,event_date
0,2019-10-01 06:00:00,view,44600062,unknown,shiseido,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,2019-10-01


In [47]:
%%time
# 월 컬럼 생성
df_oct['event_month'] = df_oct['event_time'].dt.month
df_nov['event_month'] = df_nov['event_time'].dt.month
df_oct.head(1)

Wall time: 9.36 s


,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,event_date,event_month
0,2019-10-01 06:00:00,view,44600062,unknown,shiseido,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,2019-10-01,10


In [48]:
%%time
# 일 컬럼 생성
df_oct['event_day'] = df_oct['event_time'].dt.day
df_nov['event_day'] = df_nov['event_time'].dt.day
df_oct.head(1)

Wall time: 11.3 s


,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,event_date,event_month,event_day
0,2019-10-01 06:00:00,view,44600062,unknown,shiseido,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,2019-10-01,10,1


In [49]:
%%time
# 요일 생성
df_oct['day_of_week'] = df_oct['event_time'].dt.day_name()
df_nov['day_of_week'] = df_nov['event_time'].dt.day_name()
df_oct.head(1)

Wall time: 45.2 s


,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,event_date,event_month,event_day,day_of_week
0,2019-10-01 06:00:00,view,44600062,unknown,shiseido,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,2019-10-01,10,1,Tuesday


In [50]:
%%time
# 시 생성
df_oct['event_hour'] = df_oct['event_time'].dt.hour
df_nov['event_hour'] = df_nov['event_time'].dt.hour
df_oct.head(1)

Wall time: 10.3 s


,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,event_date,event_month,event_day,day_of_week,event_hour
0,2019-10-01 06:00:00,view,44600062,unknown,shiseido,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,2019-10-01,10,1,Tuesday,6


In [51]:
%%time
# 주차 컬럼 생성
df_oct['event_week'] = df_oct['event_time'].dt.isocalendar()['week']
df_nov['event_week'] = df_nov['event_time'].dt.isocalendar()['week']
df_oct.head(1)

Wall time: 28.9 s


,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,event_date,event_month,event_day,day_of_week,event_hour,event_week
0,2019-10-01 06:00:00,view,44600062,unknown,shiseido,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,2019-10-01,10,1,Tuesday,6,40


In [52]:
df_oct.dtypes

event_time       datetime64[ns]
event_type             category
product_id                int32
category_code          category
brand                  category
price                   float32
user_id                   int32
user_session           category
event_date               object
event_month               int64
event_day                 int64
day_of_week              object
event_hour                int64
event_week               UInt32
dtype: object

In [53]:
%%time
# 추가된 열의 형변환
# category 변환 - 요일
df_oct['day_of_week'] = df_oct['day_of_week'].astype('category')
df_nov['day_of_week'] = df_nov['day_of_week'].astype('category')

# # int32 변환
df_oct[['event_month','event_day','event_hour']] = df_oct[['event_month','event_day','event_hour']].astype('int32')
df_nov[['event_month','event_day','event_hour']] = df_nov[['event_month','event_day','event_hour']].astype('int32')

Wall time: 24.8 s


In [54]:
df_oct.dtypes

event_time       datetime64[ns]
event_type             category
product_id                int32
category_code          category
brand                  category
price                   float32
user_id                   int32
user_session           category
event_date               object
event_month               int32
event_day                 int32
day_of_week            category
event_hour                int32
event_week               UInt32
dtype: object

- 12월 ~4월

In [28]:
def cleaning_time(df):
    # 시간대 +6 (카자흐스탄 시간으로 바꿔주기)
    df['event_time'] = pd.DatetimeIndex(df['event_time']) + timedelta(hours=6)
    
    # 날짜만 별도로 추출
    df['event_date'] = pd.DatetimeIndex(df['event_time']).date
        
    # 월 컬럼 생성
    df['event_month'] = df['event_time'].dt.month
    
    # 주차 컬럼 생성
    df['event_week'] = df['event_time'].dt.isocalendar()['week']
    
    return df

In [30]:
%%time
# 자꾸 ram 부족 터져서 한 줄에 하나씩
# 전처리 진행
cleaning_time(df_dec)

Wall time: 33.7 s


,event_time,event_type,category_code,price,user_id,event_date,event_month,event_week
0,2019-12-01 06:00:00,view,construction.tools.light,1302.479980,556695836,2019-12-01,12,48
1,2019-12-01 06:00:00,view,unknown,102.959999,577702456,2019-12-01,12,48
2,2019-12-01 06:00:01,view,appliances.personal.massager,313.519989,539453785,2019-12-01,12,48
3,2019-12-01 06:00:02,purchase,computers.peripherals.printer,132.309998,535135317,2019-12-01,12,48
4,2019-12-01 06:00:02,view,apparel.trousers,101.680000,517987650,2019-12-01,12,48
...,...,...,...,...,...,...,...,...
67542873,2020-01-01 05:59:58,view,construction.tools.light,908.799988,515771654,2020-01-01,1,1
67542874,2020-01-01 05:59:58,view,electronics.audio.headphone,1044.250000,595160289,2020-01-01,1,1
67542875,2020-01-01 05:59:59,view,kids.dolls,411.829987,526621231,2020-01-01,1,1
67542876,2020-01-01 05:59:59,view,sport.bicycle,17.990000,574723072,2020-01-01,1,1


In [31]:
cleaning_time(df_jan)

,event_time,event_type,category_code,price,user_id,event_date,event_month,event_week
0,2020-01-01 06:00:00,view,construction.tools.light,1130.020020,519698804,2020-01-01,1,1
1,2020-01-01 06:00:01,view,construction.tools.light,205.669998,527767423,2020-01-01,1,1
2,2020-01-01 06:00:01,view,apparel.shirt,136.429993,519046195,2020-01-01,1,1
3,2020-01-01 06:00:01,view,apparel.shoes,29.950001,518269232,2020-01-01,1,1
4,2020-01-01 06:00:02,view,apparel.shoes.step_ins,167.199997,587748686,2020-01-01,1,1
...,...,...,...,...,...,...,...,...
55967036,2020-02-01 05:59:59,view,apparel.shoes,33.720001,517548219,2020-02-01,2,5
55967037,2020-02-01 05:59:59,view,construction.tools.light,228.830002,512985520,2020-02-01,2,5
55967038,2020-02-01 05:59:59,view,apparel.shoes,47.360001,513043451,2020-02-01,2,5
55967039,2020-02-01 05:59:59,view,furniture.bedroom.blanket,51.299999,581011571,2020-02-01,2,5


In [32]:
cleaning_time(df_feb)

,event_time,event_type,category_code,price,user_id,event_date,event_month,event_week
0,2020-02-01 06:00:00,view,apparel.shoes,19.309999,581521975,2020-02-01,2,5
1,2020-02-01 06:00:00,view,sport.bicycle,11.880000,596313635,2020-02-01,2,5
2,2020-02-01 06:00:00,view,construction.tools.light,838.229980,608821035,2020-02-01,2,5
3,2020-02-01 06:00:01,view,construction.tools.light,410.420013,530488542,2020-02-01,2,5
4,2020-02-01 06:00:01,view,appliances.kitchen.refrigerators,332.029999,583795643,2020-02-01,2,5
...,...,...,...,...,...,...,...,...
55318560,2020-03-01 05:59:57,view,construction.tools.welding,1.930000,519011545,2020-03-01,3,9
55318561,2020-03-01 05:59:58,view,appliances.kitchen.toster,77.199997,540823940,2020-03-01,3,9
55318562,2020-03-01 05:59:58,view,electronics.telephone,49.910000,531574188,2020-03-01,3,9
55318563,2020-03-01 05:59:59,view,electronics.camera.video,10.770000,614934914,2020-03-01,3,9


In [33]:
cleaning_time(df_mar)

,event_time,event_type,category_code,price,user_id,event_date,event_month,event_week
0,2020-03-01 06:00:00,view,appliances.kitchen.mixer,117.629997,550295099,2020-03-01,3,9
1,2020-03-01 06:00:01,view,apparel.underwear,264.369995,622070532,2020-03-01,3,9
2,2020-03-01 06:00:03,view,furniture.bedroom.bed,126.099998,552832344,2020-03-01,3,9
3,2020-03-01 06:00:03,view,construction.tools.generator,424.720001,613391860,2020-03-01,3,9
4,2020-03-01 06:00:04,view,construction.tools.light,482.730011,517021211,2020-03-01,3,9
...,...,...,...,...,...,...,...,...
56341236,2020-04-01 05:59:55,view,construction.tools.light,178.119995,516506125,2020-04-01,4,14
56341237,2020-04-01 05:59:55,view,appliances.personal.massager,154.419998,572260610,2020-04-01,4,14
56341238,2020-04-01 05:59:56,view,unknown,167.289993,523607693,2020-04-01,4,14
56341239,2020-04-01 05:59:56,view,appliances.personal.massager,163.160004,629229484,2020-04-01,4,14


In [34]:
cleaning_time(df_apr)

,event_time,event_type,category_code,price,user_id,event_date,event_month,event_week
0,2020-04-01 06:00:00,view,apparel.shoes.slipons,230.380005,568984877,2020-04-01,4,14
1,2020-04-01 06:00:01,view,electronics.audio.headphone,1352.670044,514955500,2020-04-01,4,14
2,2020-04-01 06:00:01,view,appliances.kitchen.refrigerators,1184.050049,633645770,2020-04-01,4,14
3,2020-04-01 06:00:02,view,electronics.audio.headphone,1724.339966,564933778,2020-04-01,4,14
4,2020-04-01 06:00:03,view,apparel.scarf,25.049999,530206135,2020-04-01,4,14
...,...,...,...,...,...,...,...,...
66589263,2020-05-01 05:59:59,view,construction.tools.light,900.900024,648021639,2020-05-01,5,18
66589264,2020-05-01 05:59:59,view,construction.tools.light,1599.780029,649775363,2020-05-01,5,18
66589265,2020-05-01 05:59:59,view,apparel.shirt,12.840000,639250275,2020-05-01,5,18
66589266,2020-05-01 05:59:59,view,appliances.kitchen.toster,1956.030029,522762529,2020-05-01,5,18


## 10월 끝에 11월 1~3일 데이터 붙이기
- 10월 4주차 데이터 확보 위해서 (11월 초반)

In [55]:
# tmp변수에 11월 1일,2일,3일 데이터 지정
tmp = df_nov[(df_nov['event_time'] < '2019-11-04')]
tmp

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,event_date,event_month,event_day,day_of_week,event_hour,event_week
0,2019-11-01 06:00:00,view,1003461,electronics.smartphone,xiaomi,489.070007,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33,2019-11-01,11,1,Friday,6,44
1,2019-11-01 06:00:00,view,5000088,appliances.sewing_machine,janome,293.649994,530496790,8e5f4f83-366c-4f70-860e-ca7417414283,2019-11-01,11,1,Friday,6,44
2,2019-11-01 06:00:01,view,17302664,unknown,creed,28.309999,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387,2019-11-01,11,1,Friday,6,44
3,2019-11-01 06:00:01,view,3601530,appliances.kitchen.washer,lg,712.869995,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f,2019-11-01,11,1,Friday,6,44
4,2019-11-01 06:00:01,view,1004775,electronics.smartphone,xiaomi,183.270004,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2,2019-11-01,11,1,Friday,6,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4324040,2019-11-03 23:59:59,view,1004718,electronics.smartphone,haier,77.199997,561765733,bafdd7cd-77be-42f7-8eb4-d01ec78a1703,2019-11-03,11,3,Sunday,23,44
4324041,2019-11-03 23:59:59,view,28716595,apparel.shoes,baden,53.799999,515166728,3ed362b8-478e-4b1a-9d07-4bb078baca29,2019-11-03,11,3,Sunday,23,44
4324042,2019-11-03 23:59:59,view,28718914,apparel.shoes.keds,nexpero,71.820000,513822823,7a190371-70cf-4026-a140-e97468fed40d,2019-11-03,11,3,Sunday,23,44
4324043,2019-11-03 23:59:59,view,22700541,unknown,unknown,14.160000,567089454,cf668a7f-19ed-4ec5-89d1-da8b755553ff,2019-11-03,11,3,Sunday,23,44


In [56]:
%%time
# 10월 데이터 마지막 부분에 tmp concat
df_oct = pd.concat([df_oct, tmp], axis=0).reset_index(drop=True)
df_oct

Wall time: 1min 34s


,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,event_date,event_month,event_day,day_of_week,event_hour,event_week
0,2019-10-01 06:00:00,view,44600062,unknown,shiseido,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,2019-10-01,10,1,Tuesday,6,40
1,2019-10-01 06:00:00,view,3900821,appliances.environment.water_heater,aqua,33.200001,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,2019-10-01,10,1,Tuesday,6,40
2,2019-10-01 06:00:01,view,17200506,furniture.living_room.sofa,unknown,543.099976,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,2019-10-01,10,1,Tuesday,6,40
3,2019-10-01 06:00:01,view,1307067,computers.notebook,lenovo,251.740005,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,2019-10-01,10,1,Tuesday,6,40
4,2019-10-01 06:00:04,view,1004237,electronics.smartphone,apple,1081.979980,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,2019-10-01,10,1,Tuesday,6,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46772804,2019-11-03 23:59:59,view,1004718,electronics.smartphone,haier,77.199997,561765733,bafdd7cd-77be-42f7-8eb4-d01ec78a1703,2019-11-03,11,3,Sunday,23,44
46772805,2019-11-03 23:59:59,view,28716595,apparel.shoes,baden,53.799999,515166728,3ed362b8-478e-4b1a-9d07-4bb078baca29,2019-11-03,11,3,Sunday,23,44
46772806,2019-11-03 23:59:59,view,28718914,apparel.shoes.keds,nexpero,71.820000,513822823,7a190371-70cf-4026-a140-e97468fed40d,2019-11-03,11,3,Sunday,23,44
46772807,2019-11-03 23:59:59,view,22700541,unknown,unknown,14.160000,567089454,cf668a7f-19ed-4ec5-89d1-da8b755553ff,2019-11-03,11,3,Sunday,23,44


In [57]:
%%time
# 11월 데이터에서 tmp drop
df_nov = df_nov.drop(tmp.index, axis=0).reset_index(drop=True)
df_nov

Wall time: 54.3 s


,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,event_date,event_month,event_day,day_of_week,event_hour,event_week
0,2019-11-04 00:00:00,view,26405148,unknown,sokolov,696.799988,386908545,657eee62-2b56-40ee-be6e-63bb05623727,2019-11-04,11,4,Monday,0,45
1,2019-11-04 00:00:00,view,55000008,apparel.costume,nike,111.949997,530192239,2fb6b95e-3f82-4a44-966c-19c52954a977,2019-11-04,11,4,Monday,0,45
2,2019-11-04 00:00:00,view,35900057,unknown,apollo,10.270000,567185457,c44929e6-88db-4b73-a3d9-dbab1602159b,2019-11-04,11,4,Monday,0,45
3,2019-11-04 00:00:00,view,1201413,electronics.tablet,nexttab,47.860001,546059044,bce815d4-0e5a-40f9-ba55-688fedec3b14,2019-11-04,11,4,Monday,0,45
4,2019-11-04 00:00:00,view,12705201,unknown,unknown,58.529999,519027485,7f561f0e-3cef-4594-84c0-8e1383c0d872,2019-11-04,11,4,Monday,0,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63177929,2019-12-01 05:59:58,view,15700137,unknown,unknown,277.739990,532714000,02b4131c-0112-4231-aafa-ceaa08e77c1b,2019-12-01,12,1,Sunday,5,48
63177930,2019-12-01 05:59:58,view,28719425,apparel.shoes,baden,62.810001,545223467,734c5eef-0742-4f8b-9d22-48f75b0bc359,2019-12-01,12,1,Sunday,5,48
63177931,2019-12-01 05:59:59,view,1004833,electronics.smartphone,samsung,167.029999,557794415,6fecf566-ebb0-4e70-a243-cdc13ce044cb,2019-12-01,12,1,Sunday,5,48
63177932,2019-12-01 05:59:59,view,2701706,appliances.kitchen.refrigerators,samsung,566.270020,531607492,368ddc8b-5db9-40fb-b7ff-b6582a1192c0,2019-12-01,12,1,Sunday,5,48


In [58]:
tmp = df_nov[(df_nov['event_month'] == 12)]
tmp

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,event_date,event_month,event_day,day_of_week,event_hour,event_week
62935187,2019-12-01 00:00:00,view,1004836,electronics.smartphone,samsung,229.589996,569472327,aeaec12c-e61a-4441-bf0c-2d57345474fb,2019-12-01,12,1,Sunday,0,48
62935188,2019-12-01 00:00:00,view,11400258,unknown,remax,33.439999,512557400,c4b6a648-cf78-4ce6-99a6-675c43b77d80,2019-12-01,12,1,Sunday,0,48
62935189,2019-12-01 00:00:00,view,1307545,computers.notebook,lenovo,244.250000,529142459,6e28b9ce-a8d3-40c1-b8cc-bbd3164ccc86,2019-12-01,12,1,Sunday,0,48
62935190,2019-12-01 00:00:00,view,1004768,electronics.smartphone,samsung,236.759995,517400544,1713438a-b29c-4cc6-b416-853a1e888248,2019-12-01,12,1,Sunday,0,48
62935191,2019-12-01 00:00:00,view,12715113,unknown,kormoran,66.930000,579738611,5c0e70ac-3d8d-4a64-be47-7d608b4cdb94,2019-12-01,12,1,Sunday,0,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63177929,2019-12-01 05:59:58,view,15700137,unknown,unknown,277.739990,532714000,02b4131c-0112-4231-aafa-ceaa08e77c1b,2019-12-01,12,1,Sunday,5,48
63177930,2019-12-01 05:59:58,view,28719425,apparel.shoes,baden,62.810001,545223467,734c5eef-0742-4f8b-9d22-48f75b0bc359,2019-12-01,12,1,Sunday,5,48
63177931,2019-12-01 05:59:59,view,1004833,electronics.smartphone,samsung,167.029999,557794415,6fecf566-ebb0-4e70-a243-cdc13ce044cb,2019-12-01,12,1,Sunday,5,48
63177932,2019-12-01 05:59:59,view,2701706,appliances.kitchen.refrigerators,samsung,566.270020,531607492,368ddc8b-5db9-40fb-b7ff-b6582a1192c0,2019-12-01,12,1,Sunday,5,48


In [59]:
%%time
# 11월 데이터에서 tmp drop
df_nov = df_nov.drop(tmp.index, axis=0).reset_index(drop=True)
df_nov

Wall time: 24.5 s


,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,event_date,event_month,event_day,day_of_week,event_hour,event_week
0,2019-11-04 00:00:00,view,26405148,unknown,sokolov,696.799988,386908545,657eee62-2b56-40ee-be6e-63bb05623727,2019-11-04,11,4,Monday,0,45
1,2019-11-04 00:00:00,view,55000008,apparel.costume,nike,111.949997,530192239,2fb6b95e-3f82-4a44-966c-19c52954a977,2019-11-04,11,4,Monday,0,45
2,2019-11-04 00:00:00,view,35900057,unknown,apollo,10.270000,567185457,c44929e6-88db-4b73-a3d9-dbab1602159b,2019-11-04,11,4,Monday,0,45
3,2019-11-04 00:00:00,view,1201413,electronics.tablet,nexttab,47.860001,546059044,bce815d4-0e5a-40f9-ba55-688fedec3b14,2019-11-04,11,4,Monday,0,45
4,2019-11-04 00:00:00,view,12705201,unknown,unknown,58.529999,519027485,7f561f0e-3cef-4594-84c0-8e1383c0d872,2019-11-04,11,4,Monday,0,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62935182,2019-11-30 23:59:59,view,1004751,electronics.smartphone,samsung,192.770004,579737238,67787d24-25fd-4dfc-80e5-c642877cd84a,2019-11-30,11,30,Saturday,23,48
62935183,2019-11-30 23:59:59,view,4300392,unknown,vitek,23.139999,566828985,2954305a-def3-4a24-9d8a-129fe87938c4,2019-11-30,11,30,Saturday,23,48
62935184,2019-11-30 23:59:59,view,2900569,appliances.kitchen.microwave,lg,102.709999,518637498,8cb2fad3-10cc-4c14-9eb6-c03276a90738,2019-11-30,11,30,Saturday,23,48
62935185,2019-11-30 23:59:59,view,1306951,computers.notebook,apple,2329.530029,557228009,a598f21a-6710-4072-a4e1-4c9656cef9fb,2019-11-30,11,30,Saturday,23,48


In [60]:
#concat category 데이터가 object로 변경 되어 버림
#기존에 없던 카테고리 데이터가 신규로 생성되어서 오브젝트로 변한것으로 추정 됨
#기존과 카테고리가 같은 경우는 그대로 category 형태로 남아있음
df_oct.dtypes

event_time       datetime64[ns]
event_type             category
product_id                int32
category_code            object
brand                    object
price                   float32
user_id                   int32
user_session             object
event_date               object
event_month               int32
event_day                 int32
day_of_week            category
event_hour                int32
event_week               UInt32
dtype: object

In [61]:
df_oct['event_date'] = pd.to_datetime(df_oct['event_date'], format = '%Y-%m-%d')
df_nov['event_date'] = pd.to_datetime(df_nov['event_date'], format = '%Y-%m-%d')

In [62]:
df_oct[['category_code','brand','user_session']] = df_oct[['category_code','brand','user_session']].astype('category')
df_nov[['category_code','brand','user_session']] = df_nov[['category_code','brand','user_session']].astype('category')

In [63]:
df_oct.dtypes

event_time       datetime64[ns]
event_type             category
product_id                int32
category_code          category
brand                  category
price                   float32
user_id                   int32
user_session           category
event_date       datetime64[ns]
event_month               int32
event_day                 int32
day_of_week            category
event_hour                int32
event_week               UInt32
dtype: object

# Export cleaned Parquet

In [64]:
%%time
df_oct.to_parquet('../e-commerce/input/2019-Oct_cleaned.parquet', engine='pyarrow', index=False)
df_nov.to_parquet('../e-commerce/input/2019-Nov_cleaned.parquet', engine='pyarrow', index=False)

Wall time: 1min 51s


In [35]:
%%time
df_dec.to_parquet('../e-commerce/input/2019-Dec_cleaned.parquet', engine='pyarrow', index=False)
df_jan.to_parquet('../e-commerce/input/2020-Jan_cleaned.parquet', engine='pyarrow', index=False)
df_feb.to_parquet('../e-commerce/input/2020-Feb_cleaned.parquet', engine='pyarrow', index=False)
df_mar.to_parquet('../e-commerce/input/2020-Mar_cleaned.parquet', engine='pyarrow', index=False)
df_apr.to_parquet('../e-commerce/input/2020-Apr_cleaned.parquet', engine='pyarrow', index=False)

Wall time: 2min 6s


# +@ 전처리 for 추천시스템

In [2]:
# 전처리된 데이터셋 불러오기
df_oct = pd.read_parquet('../e-commerce/input/2019-Oct_cleaned.parquet')

ArrowMemoryError: malloc of size 187091264 failed

### 📌 가격이 0이하인 데이터 제거하기

In [ ]:
# price가 0 이하인 경우는 비정상적인 캐아스이고, 이런 데이터가 존재하는지 먼저 알아봄
print('10월 데이터 중 price가 0 이하인 데이터 수: ', df_oct[df_oct['price']<=0].price.count())

In [ ]:
# 데이터 제거하기
df_oct = df_oct[df_oct['price'] > 0]

### 📌 제거 후 데이터셋 크기 및 컬럼 타입 확인

In [ ]:
# 제거 후 데이터셋 크기 및 컬럼 타입 확인
print(df_oct.shape)
df_oct.head(3)

In [4]:
print('10월 데이터 타입 확인: \n', df_oct.dtypes)

NameError: name 'df_oct' is not defined

### 📌  컬럼 타입 변환
- `category_code`: `category -> str`
- `brand`: `category -> str`

In [8]:
# category_code, brand: vectorization을 위해 타입 문자열로 변환
df_oct['category_code'] = df_oct['category_code'].astype('str')
df_oct['brand'] = df_oct['brand'].astype('str')
df_oct['product_id'] = df_oct['product_id'].astype('str')

In [9]:
print('10월 데이터 타입 확인(타입변환 후): \n', df_oct.dtypes)

10월 데이터 타입 확인(타입변환 후): 
 event_time       datetime64[ns]
event_type             category
product_id                int32
category_code            object
brand                    object
price                   float32
user_id                   int32
user_session           category
event_date       datetime64[ns]
event_month               int32
event_day                 int32
day_of_week            category
event_hour                int32
event_week               UInt32
dtype: object


### 📌 unknown이라 표기된 결측치 제거
- cateogory_code, brand가 unknown이면 추천 시스템에 사용할 수 없으므로 제거

In [10]:
# unknown이라 표기된 결측치 제거 (추천에 방해된다고 판단)

# 1) category_code == unknown인 경우의 인덱스 보여주는 조건문
cat_con = df_oct[df_oct['category_code'] == 'unknown'].index        
df_oct.drop(cat_con, inplace=True)          # 제거

# 2) brand == unknown인 경우의 인덱스 보여주는 조건문
brand_con = df_oct[df_oct['brand'] == 'unknown'].index              
df_oct.drop(brand_con, inplace=True)        # 제거

###  ✅ 필요한 전처리를 거친 데이터를 다시 parquet 파일로 저장하기
- 이후 과정을 편하게 하기 위해 전처리를 거친 데이터 파일 따로 생성해 나중에 로드하려 함

In [11]:
# 내보내기
df_oct.to_parquet('../e-commerce/input/2019-Oct_new.parquet', engine='pyarrow', index=False)